In [1]:
import os
import shutil
from pathlib import Path
from HandsFreeDocking.Wrapper_Docking import PipelineDocking
from HandsFreeDocking.tools.Ligand_Preparation import LigandPreparator

In [2]:
protein_pdb = Path("./examples/LAG3_Moloc_2.pdb")
ligands_sdf = Path("./examples/Ligands_To_Dock.sdf")
cystal_sdf = Path("./examples/Fake_Crystal.sdf")

docking_dir = Path("./examples/TMP_Docking")

In [3]:
if docking_dir.exists():
    shutil.rmtree(docking_dir, ignore_errors=True)

In [4]:
docking = PipelineDocking(
    workdir=docking_dir,
    docking_software=["plants"],
    settings=(10, 4),
    protein_pdb=protein_pdb,
    ligands_input=ligands_sdf,
    crystal_sdf=cystal_sdf,
    protonation_method="cdp"
)

In [5]:
results = docking.run()
FULL_DF = docking.concat_df()

2025-05-31 22:47:02,685 - HandsFreeDocking.Wrapper_Docking - INFO - Using SDF input directly: examples/Ligands_To_Dock.sdf
2025-05-31 22:47:02,688 - HandsFreeDocking.Wrapper_Docking - INFO - Starting Plants docking


Preparing protein with Chimera ...


2025-05-31 22:47:04,766 - HandsFreeDocking.Plants_Pipeline - INFO - Preparing ligands using cdp protonation method
2025-05-31 22:47:04,767 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Apigenin from SDF
2025-05-31 22:47:04,870 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing 6-methylflavone from SDF
2025-05-31 22:47:04,897 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Chrysin from SDF
2025-05-31 22:47:04,939 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Luteolin from SDF
2025-05-31 22:47:05,149 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Kaempferol from SDF
2025-05-31 22:47:05,307 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Eriodictyol from SDF
2025-05-31 22:47:05,543 - HandsFreeDocking.tools.Ligand_Preparation - INFO - Processing Fisetin from SDF
2025-05-31 22:47:05,717 - HandsFreeDocking.Plants_Pipeline - INFO - Storing template SMILES mapping for MOL2 fixing
2025-05-31 22:47:

## Ligand Preparation

In [ ]:
smiles_to_test = [
    "C1C(C[NH3+])=CC2=C(C(N[C@@H](C3C=CC=C(C(=O)[O-])C=3)C)=NS2(=O)=O)C=1",
    "C1C(C[NH3+])=CC2=C(C(NC(C3C=CC=C(C(=O)[O-])C=3)C)=NS2(=O)=O)C=1",
    "N1C=C(C2CC2)C(=O)NC1=O",
    "C1C=CN2C(NC(C)C)=C(CC)N=C2C=1",
    "C1C=CN2C(NC(C)C)=CN=C2C=1",
    "C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O"
]

names_of_smiles = [
    "NUM_1",
    "NUM_2",
    "NUM_3",
    "NUM_4",
    "NUM_5",
    "NUM_6"
]

assert len(smiles_to_test) == len(names_of_smiles), "The number of SMILES and names must match."

In [ ]:
preparator = LigandPreparator(
    protonation_method="cdp",
    enumerate_stereo=True,
    tautomer_score_threshold=None,
    generate_3d=True
)

# Prepare molecules from SDF
prepared_mols = preparator.prepare_from_smiles(smiles_to_test, names_of_smiles)
print(len(prepared_mols), "molecules prepared from SMILES.")

## MOL2 -> SDF Conv

In [ ]:
import os
import shutil
from pathlib import Path
from HandsFreeDocking.Wrapper_Docking import PipelineDocking
from HandsFreeDocking.tools.Ligand_Preparation import LigandPreparator

In [ ]:
from HandsFreeDocking.tools.Fix_Mol2 import hard_fix_mol2
from rdkit import Chem
from rdkit.Chem import AllChem
import datamol as dm
from typing import List, Tuple, Optional

In [ ]:
mol2_example = Path("examples/TMP_Docking/Plants/output_plants/Apigenin_Iso0/docked_ligands.mol2")
assert mol2_example.exists(), f"Expected {mol2_example} to exist."

smiles_example = "C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O"

In [ ]:
fixed_mols = hard_fix_mol2(mol2_example, smiles_example)